<a href="https://colab.research.google.com/github/inspectorgadgetteknofest/inspector-gadget-acikhack2021/blob/main/Osmanl%C4%B1ca/Osmanl%C4%B1ca_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kütüphanelerin Import Edilmesi:

In [ ]:
# Drive'ı initialize etme
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Bleu score hesaplamak için
!pip install sacrebleu

In [ ]:
!pip install transformers
!pip install simpletransformers
# !pip install wandb -q

In [ ]:
# Veriyi çekme ve ön işleme için:
import pandas as pd

# Veriyi train2e hazırlama ve train etme için
from sklearn.model_selection import train_test_split
import logging
from simpletransformers.t5 import T5Model, T5Args

# Predict ve Bleu Score hesaplamak için 
import sacrebleu

## 2. Veri Seti


In [ ]:
# Verisetinin okunması:
data_path = "oe_tr.xlsx" #@param {type:"string"}
df = pd.read_excel(data_path)

# Veri setinin incelenmesi
df

## Osmanlıca-Türkçe mi Türkçe-Osmanlıca mı?

Kullanılacak çeviri işlemine göre ikisinden biri çalıştırılmalıdır.

Osmanlıca --> Türkçe ise:

In [ ]:
# Sütun adlarının değiştirilmesi
df.columns = ['input_text','target_text']

# İstenilen dile çevirilirken 'translate' kelimesi eklendi.
df.target_text = 'translate: ' + df.target_text

# İlk 5 veriyi görelim:
print(df.head())

Türkçe --> Osmanlıca ise:

In [ ]:
# Sütun adlarının değiştirilmesi:
df.columns = ['target_text','input_text']

# İstenilen dile çevirilirken 'translate' kelimesi eklendi.
df.target_text = 'translate: ' + df.target_text

# İlk 5 veriyi görelim:
print(df.head())

In [ ]:
# Boş bir sütun ekleyip adını 'prefix' verelim. Bu sütun modele birden fazla işlem 
# tanımlandığı durumlarda kullanılır. Bizde tek bir işlem olacağı için bu sütunun
# büyük bir anlamı olmayacak ama yine de koyalım : 
df["prefix"] = ""
df

In [ ]:
# Train test ayrımını yapalım. (Bunu yaparken 0.2 oranında ayıralım)
train, test = train_test_split(df, test_size = 0.20, random_state = 42)

In [ ]:
# train veri setine göz atalım:
train

In [ ]:
# test veri setine göz atalım:
test

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# Modelin kaydedileceği path bilgisi :
OUTPUT_DIR = "osmanl\u0131ca-tu\u0308rkc\u0327e/model" #@param {type:"string"}

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 256
model_args.train_batch_size = 4
model_args.eval_batch_size = 4
model_args.num_train_epochs = 3
model_args.evaluate_during_training = False
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = 5000
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = True
model_args.output_dir = OUTPUT_DIR
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_return_sequences = 1
#model_args.wandb_project = "MT5 mixed tasks"

model = T5Model("mt5", "google/mt5-base", args=model_args)

In [ ]:
# Model eğitimi:
model.train_model(train)

In [ ]:
# Modelin tahmin ettirilmesi:
model.predict(["Atılmış"])

## Predict ve Evaluation Kısmı:

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 256
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "./checkpoint-5000", args=model_args)

In [ ]:
# Birbirleriyle karşılaştırmak için tahminleri ve doğru değerleri bir listeye atalım:
eval = test["input_text"].to_list()
true = test["target_text"].to_list()

In [ ]:
# Tahminleri bir yerde toplayalım:
preds = model.predict(eval)

# bleu score'u ölçmek için: 
bleu = sacrebleu.corpus_bleu(preds, true)
print("--------------------------")
print(f"Türkçe'den Osmanlıca'ya: " {bleu.score})

## Eğitimin Sonuçları:

In [ ]:
# Eğitimin bleu score'unun hesaplanması:
bleu = sacrebleu.corpus_bleu(preds, true)

In [ ]:
# bleu score'unun ekrana bastırılması:
bleu.score

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("./checkpoint-5000")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Açık Hack Yarışma/osmanlıca-türkçe/model/checkpoint-5000").cpu()
def translate(text):
  translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
  translation= [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  return translation


In [ ]:
def translate_text(estimator, subtokenizer, txt):
  """Tek bir text i translate etmek için"""
  encoded_txt = _encode_and_add_eos(txt, subtokenizer)

  def input_fn():
    ds = tf.data.Dataset.from_tensors(encoded_txt)
    ds = ds.batch(_DECODE_BATCH_SIZE)
    return ds

  predictions = estimator.predict(input_fn)
  translation = next(predictions)["outputs"]
  translation = _trim_and_decode(translation, subtokenizer)
  tf.logging.info("Bunun çevirisi \"%s\": \"%s\"" % (txt, translation))
  return translation

In [ ]:
import tensorflow as tf
import FastPredict

class FastPredict:

    def __init__(self, estimator, input_fn):
        self.estimator = estimator
        self.first_run = True
        self.closed = False
        self.input_fn = input_fn

    def _create_generator(self):
        while not self.closed:
            yield self.next_features

    def predict(self, feature_batch):
        
        self.next_features = feature_batch
        if self.first_run:
            self.batch_size = len(feature_batch)
            self.predictions = self.estimator.predict(
                input_fn=self.input_fn(self._create_generator))
            self.first_run = False
        elif self.batch_size != len(feature_batch):
            raise ValueError("Tüm batchler eşit uzunlukta olmalıdır. İlk-batch:" + str(self.batch_size) + " Bu-batch:" + str(len(feature_batch)))

        results = []
        for _ in range(self.batch_size):
            results.append(next(self.predictions))
        return results

    def close(self):
        self.closed = True
        try:
            next(self.predictions)
        except:
            print("Exception in predict. This is probably OK")


def example_input_fn(generator):
    

    def _inner_input_fn():
        dataset = tf.data.Dataset().from_generator(generator, output_types=(tf.float32)).batch(1)
        iterator = dataset.make_one_shot_iterator()
        features = iterator.get_next()
        return {'x': features}

    return _inner_input_fn
classifier = FastPredict(learn.Estimator(model_fn=model_params.model_fn, model_dir=model_params.model_dir), my_input_fn)